In [1]:
import requests
import json
import pandas as pd

In [2]:
with open('keyfile') as keyfile:
    API_KEY= keyfile.read()
    
## Geography
london_box = (51.145756,-0.913796,51.955268,0.388083)
london_mid = (51.5,-0.12)
# Trafalgar Square
uk_box = (49.3739906, -7.0113373, 61.2291139, 2.4798203)
# Isles of Scilly to Shetland Islands (includes a bit of belgium and most of ROI)

# Requests
url = 'https://airlabs.co/api/v9/'
endpoint = lambda e: url + e

# Get Static Data
with open('airports.json') as airport_data:
    airports = json.load(airport_data)
    
with open('ldn_airports.json') as ldn_airports_file:
    ldn_airports = json.load(ldn_airports_file)
    
# Make it useful
codes = {ap['iata_code']:ap['name'] for ap in airports if 'iata_code' in ap.keys()}

In [3]:
# Request live data

r= requests.get(endpoint('flights'),
                params={
                    'api_key':API_KEY,
                    'bbox':london_box,
                }
               )
all_planes = r.json()['response']
flying_planes = [plane for plane in all_planes if plane['status'] == 'en-route']

# make DataFrame

df= pd.DataFrame(flying_planes)
#print(df.columns); df.head()

# Make the dataframe more user friendly

df['Source'] = df['dep_iata'].map(lambda x : codes[x], 
                                  na_action='ignore')
df['Destination'] = df['arr_iata'].map(lambda x : codes[x], 
                                       na_action='ignore')

# Divide frames into arrivals, departures, fly-overs, mystery and intra-london (naughty) flights
df_to_london = df[df['Destination'].isin([a['name'] for a in ldn_airports])]
df_from_london = df[df['Source'].isin([a['name'] for a in ldn_airports])]
df_flyover = df[-df['Source'].isin([a['name'] for a in ldn_airports])][-df['Destination'].isin([a['name'] for a in ldn_airports])].dropna()
df_mystery = df[df['Source'].isna()]
df_naughty = df_to_london[df_to_london['Source'].isin([a['name'] for a in ldn_airports])]

# cheeky Lambda for representing the dataframes
show_frame = lambda df : df[['Source','Destination','alt','reg_number','flag']].sort_values(['alt','Destination'])

/var/folders/0f/wlc6r7s15q1fjgdnp6nzb3kc0000gp/T/ipykernel_32198/4221847234.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_flyover = df[-df['Source'].isin([a['name'] for a in ldn_airports])][-df['Destination'].isin([a['name'] for a in ldn_airports])].dropna()


### If there are any planes flying between london airports AND they've put this in their trackers, they will show up here:

In [9]:
if len(df_naughty)>0:
    show_frame(df_naughty)
else:
    print("no planes")

no planes


In [5]:
show_frame(df_flyover)

,Source,Destination,alt,reg_number,flag
7,Wroclaw-Copernicus Airport,Bournemouth Airport,10728,SP-RSU,IE
5,Brussels Airport,Birmingham International Airport,10972,OO-SSS,BE
6,Amsterdam Airport Schiphol,Grantley Adams International Airport,10972,PH-AOA,NL
4,Malta International Airport,Manchester Airport,11323,G-RUKD,UK
2,Aeroport Geneve-Cointrin,Manchester Airport,11582,G-EZOP,UK
1,Frankfurt-Hahn Airport,Kerry Airport,12184,EI-GSH,IE


In [6]:
show_frame(df_from_london)

,Source,Destination,alt,reg_number,flag
79,London Heathrow Airport,Los Angeles International Airport,-99,N727AN,US
75,London Stansted Airport,Dubai International Airport,-22,A6-EQJ,AE
29,London Heathrow Airport,Aeroport Geneve-Cointrin,0,G-EUYR,UK
11,London Heathrow Airport,Bahrain International Airport,0,G-VIIW,UK
9,London Gatwick Airport,Barcelona-El Prat Airport,0,EC-LOB,ES
20,London Heathrow Airport,Budapest Ferenc Liszt International Airport,0,G-EUUB,UK
13,London Heathrow Airport,Lisbon Humberto Delgado Airport,0,CS-TJP,PT
34,London Heathrow Airport,Norman Y. Mineta San Jose International Airport,0,G-ZBKC,UK
17,London Heathrow Airport,Philadelphia International Airport,0,G-VIIC,UK
21,London Heathrow Airport,Toulouse-Blagnac Airport,0,G-EUUK,UK


In [7]:
show_frame(df_to_london)

,Source,Destination,alt,reg_number,flag
33,Lyon-Saint Exupery Airport,London Gatwick Airport,0,G-GATR,UK
14,Hamad International Airport,London Heathrow Airport,0,A7-BFD,QA
25,John Paul II International Airport Krakow-Balice,London Heathrow Airport,0,G-EUUS,UK
74,Bahrain International Airport,London Heathrow Airport,0,A9C-FB,BH
10,Adolfo Suarez Madrid-Barajas Airport,London Heathrow Airport,91,EC-NTA,ES
38,Edinburgh Airport,London Heathrow Airport,480,G-NEOR,UK
19,Nnamdi Azikiwe International Airport,London Heathrow Airport,914,G-RAES,UK
69,Milano Malpensa Airport,London Stansted Airport,998,9H-QBH,IE
28,Francisco de Sa Carneiro Airport,London Gatwick Airport,1264,G-EZGN,GB
80,El Dorado International Airport,London Heathrow Airport,1303,N792AV,CO
